In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import uproot

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))


In [ ]:
from src.pyroot_loading import get_rw_sys_weights_dic

In [ ]:
ntuple_energies = uproot.open("../small_data_files/random_ncpi0_ntuple_file.root")["wcpselection"]["T_KINEvars"].arrays(["kine_reco_Enu"], library="np")["kine_reco_Enu"]
ntuple_rses = uproot.open("../small_data_files/random_ncpi0_ntuple_file.root")["wcpselection"]["T_eval"].arrays(["run", "subrun", "event"], library="np")
print(ntuple_rses)


In [ ]:
arborist_runs = uproot.open("../small_data_files/arborist_20251216T143955_out.root")["arborist"]["eventweight_tree"]["run"].array(library="np")
arborist_subruns = uproot.open("../small_data_files/arborist_20251216T143955_out.root")["arborist"]["eventweight_tree"]["sub_run"].array(library="np")
arborist_events = uproot.open("../small_data_files/arborist_20251216T143955_out.root")["arborist"]["eventweight_tree"]["event"].array(library="np")

print(arborist_runs)
print(arborist_subruns)
print(arborist_events)


In [ ]:
spline_rses = uproot.open("../small_data_files/arborist_20251216T143955_out.root")["arborist"]

In [ ]:
spline_weight_dics = get_rw_sys_weights_dic("../small_data_files/arborist_20251216T143955_out.root", "arborist/eventweight_tree", "mcweight")

In [ ]:
print(len(spline_weight_dics))
spline_weight_dics[0]

In [ ]:
vartypes = list(spline_weight_dics[0].keys())
num_events = len(spline_weight_dics)

for vartype in vartypes:
    num_unis = len(spline_weight_dics[0][vartype])
    plt.figure()
    for uni_i in range(num_unis):
        event_weights = [spline_weight_dics[event_i][vartype][uni_i] for event_i in range(num_events)]
        event_weights = np.array(event_weights)
        event_weights[event_weights > 30] = 1
        event_weights[event_weights < 0] = 1
        event_weights[np.isnan(event_weights)] = 1
        event_weights[np.isinf(event_weights)] = 1
        colors = plt.cm.gist_rainbow(np.linspace(0, 1, num_unis))
        plt.hist(ntuple_energies, weights=event_weights, bins=np.linspace(0, 1000, 11), histtype="step", color=colors[uni_i])
    plt.title(f"{vartype} with {num_unis} universes")
    plt.xlabel("WC kine_reco_Enu (MeV)")
    plt.ylabel("Summed weight")
    plt.show()
